In [1]:
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
# Leer el archivo CSV con separador coma
df = pd.read_csv("datasets/startups_etapa_temprana.csv", sep=",")
df.head()

,company_name,link_company,equity_funding_total,num_articles,last_equity_funding_total,num_investments_funding_rounds,founded_on,ipqwery_num_trademark_registered,ipqwery_num_patent_granted,num_investors,...,num_funding_rounds,last_funding_type,last_funding_at,last_funding_total,num_founders,linkedin,facebook,twitter,founders,categories
0,QFEX,https://www.crunchbase.com/organization/qfex,500000.0,NaN,500000.0,NaN,2025-05-01,NaN,NaN,1,...,1,pre_seed,2025-05-04,500000.0,2.0,https://www.linkedin.com/company/Entimox/,NaN,https://x.com/TradeQFEX,"Annanay Kapila, Joshua Wharton","Financial Services, Lending and Investments"
1,Eytrix,https://www.crunchbase.com/organization/eytrix,29000000.0,NaN,26000000.0,NaN,2025-03-25,NaN,NaN,6,...,2,series_a,2025-05-08,26000000.0,2.0,https://www.linkedin.com/company/eytrix/,NaN,https://x.com/eytrixai,"Erdem Hacisalihoglu, Ethan Lee","Artificial Intelligence (AI), Data and Analyti..."
2,Minerva Intelligence,https://www.crunchbase.com/organization/minerv...,525000.0,NaN,525000.0,NaN,2025-02-23,NaN,NaN,3,...,1,pre_seed,2025-04-07,525000.0,1.0,NaN,NaN,NaN,Om Agarwal,Financial Services
3,Terracotta AI,https://www.crunchbase.com/organization/terrac...,2700000.0,NaN,2000000.0,NaN,2025-02-01,NaN,NaN,2,...,2,seed,2025-01-01,2000000.0,2.0,https://www.linkedin.com/company/tryterracotta,NaN,NaN,"Carlos Feliciano, Casey Wilcox","Artificial Intelligence (AI), Hardware, Inform..."
4,TripleZip,https://www.crunchbase.com/organization/triplezip,500000.0,NaN,500000.0,NaN,2025-01-05,NaN,NaN,1,...,1,pre_seed,2025-03-12,500000.0,2.0,https://www.linkedin.com/company/triplezip,NaN,https://x.com/TripleZipAI,"Grayson Pike, Yash Sahota","Artificial Intelligence (AI), Data and Analyti..."


In [3]:
len(df)

2557

In [4]:
# Leer el archivo con separador ;
fundadores_df = pd.read_csv("datasets/conteo_fundadores_por_startup.csv", sep=";")
fundadores_df.head()

,company_name,num_female_founders,num_male_founders
0,0pass,0,2
1,10by10.io,1,0
2,1849 Bio,0,2
3,1Flow,0,1
4,1build,0,1


In [5]:
# Hacer el merge (cruce) por company_name
df = df.merge(
    fundadores_df[["company_name", "num_female_founders", "num_male_founders"]],
    on="company_name",
    how="left"
)

In [6]:
len(df)

2557

In [7]:
# Crear columnas binarias sin sobrescribir las originales
df["has_linkedin"] = df["linkedin"].apply(lambda x: 1 if pd.notna(x) and str(x).strip() != "" else 0)
df["has_facebook"] = df["facebook"].apply(lambda x: 1 if pd.notna(x) and str(x).strip() != "" else 0)

In [8]:
len(df)

2557

In [9]:
# Si no tiene LinkedIn, colocar 0 en followers y empleados
df["linkedin_followers"] = df["has_linkedin"].apply(lambda x: 0 if not x else pd.NA)
df["linkedin_employees"] = df["has_linkedin"].apply(lambda x: 0 if not x else pd.NA)

# Si no tiene Facebook, colocar 0 en followers
df["facebook_followers"] = df["has_facebook"].apply(lambda x: 0 if not x else pd.NA)


In [10]:
# Reordenar columnas para insertar después de 'linkedin'
col_list = list(df.columns)
idx = col_list.index("linkedin")

# Insertar las nuevas columnas en la posición deseada
for col_name in ["linkedin_employees", "linkedin_followers"][::-1]:  # insert en orden inverso
    col_list.insert(idx + 1, col_list.pop(col_list.index(col_name)))

# Aplicar nuevo orden
df = df[col_list]

In [11]:
# Reordenar columnas para insertar después de 'linkedin'
col_list = list(df.columns)
idx = col_list.index("facebook")

# Insertar las nuevas columnas en la posición deseada
for col_name in ["facebook_followers"][::-1]:  # insert en orden inverso
    col_list.insert(idx + 1, col_list.pop(col_list.index(col_name)))

# Aplicar nuevo orden
df = df[col_list]

In [12]:
# Leer el archivo con separador ;
df_x = pd.read_csv("datasets/cuentas_startup_twitter_con_seguidores.csv", sep=";")
df_x.head()

,company_name,twitter,twitter_followers
0,QFEX,https://x.com/TradeQFEX,183
1,Novoflow,NaN,NaN
2,Frekil,NaN,NaN
3,Eytrix,https://x.com/eytrixai,0
4,Minerva Intelligence,NaN,NaN


In [13]:
len(df_x)

4669

In [14]:
#duplicados = df[df["company_name"].duplicated()]
#print(duplicados["company_name"])

In [15]:
# Hacer el merge temporalmente solo para obtener la columna de twitter
twitter_merge = pd.merge(df[["company_name"]], df_x[["company_name", "twitter"]], on="company_name", how="left")

# Crear nueva columna has_twitter directamente en df
df["has_twitter"] = twitter_merge["twitter"].apply(lambda x: 1 if pd.notna(x) and str(x).strip() != "" else 0)

# Verificamos el resultado
print(df[["company_name", "has_twitter"]].head())

           company_name  has_twitter
0                  QFEX            1
1                Eytrix            1
2  Minerva Intelligence            0
3         Terracotta AI            0
4             TripleZip            1


In [16]:
len(df)

2557

In [17]:
# Hacemos merge por 'name' para traer los followers
df = pd.merge(df, df_x[["company_name", "twitter_followers"]], on="company_name", how="left")

# Rellenar con 0 donde no haya seguidores
df["twitter_followers"] = pd.to_numeric(df["twitter_followers"], errors="coerce").fillna(0).astype(int)

# Verificamos el resultado
print(df[["company_name", "twitter_followers"]].head())

           company_name  twitter_followers
0                  QFEX                183
1                Eytrix                  0
2  Minerva Intelligence                  0
3         Terracotta AI                  0
4             TripleZip                 16


In [18]:
len(df)

2557

In [19]:
df = df.drop(["twitter", "founders"], axis=1)

In [20]:
columnas = ["num_articles", "num_investments_funding_rounds", "ipqwery_num_trademark_registered","ipqwery_num_patent_granted",] 

df[columnas] = df[columnas].replace("", pd.NA)  # convierte "" en NaN
df[columnas] = df[columnas].fillna(0)           # reemplaza NaN por 0


In [21]:
df[columnas] = df[columnas].astype("Int64")

In [22]:
len(df)

2557

In [23]:
# Leer el archivo CSV con separador coma
df_c = pd.read_csv("datasets/categorias_startups_transformadas.csv", sep=",")
df_c.head()

,company_name,categories,main_category
0,QFEX,"Financial Services, Lending and Investments",Finanzas
1,Eytrix,"Artificial Intelligence (AI), Data and Analyti...",Software y Servicios B2B
2,Minerva Intelligence,Financial Services,Finanzas
3,Terracotta AI,"Artificial Intelligence (AI), Hardware, Inform...",Software y Servicios B2B
4,TripleZip,"Artificial Intelligence (AI), Data and Analyti...",Bienes Raíces y Construcción


In [24]:
# Hacer el merge temporalmente solo para obtener la columna de twitter
main_category = pd.merge(df[["company_name"]], df_c[["company_name", "main_category"]], on="company_name", how="left")

# Crear nueva columna has_twitter directamente en df
df["category"] = main_category["main_category"]

# Verificamos el resultado
print(df[["company_name", "category"]].head())

           company_name                      category
0                  QFEX                      Finanzas
1                Eytrix      Software y Servicios B2B
2  Minerva Intelligence                      Finanzas
3         Terracotta AI      Software y Servicios B2B
4             TripleZip  Bienes Raíces y Construcción


In [25]:
df.head()

,company_name,link_company,equity_funding_total,num_articles,last_equity_funding_total,num_investments_funding_rounds,founded_on,ipqwery_num_trademark_registered,ipqwery_num_patent_granted,num_investors,...,facebook,facebook_followers,categories,num_female_founders,num_male_founders,has_linkedin,has_facebook,has_twitter,twitter_followers,category
0,QFEX,https://www.crunchbase.com/organization/qfex,500000.0,0,500000.0,0,2025-05-01,0,0,1,...,NaN,0,"Financial Services, Lending and Investments",1,1,1,0,1,183,Finanzas
1,Eytrix,https://www.crunchbase.com/organization/eytrix,29000000.0,0,26000000.0,0,2025-03-25,0,0,6,...,NaN,0,"Artificial Intelligence (AI), Data and Analyti...",0,2,1,0,1,0,Software y Servicios B2B
2,Minerva Intelligence,https://www.crunchbase.com/organization/minerv...,525000.0,0,525000.0,0,2025-02-23,0,0,3,...,NaN,0,Financial Services,1,0,0,0,0,0,Finanzas
3,Terracotta AI,https://www.crunchbase.com/organization/terrac...,2700000.0,0,2000000.0,0,2025-02-01,0,0,2,...,NaN,0,"Artificial Intelligence (AI), Hardware, Inform...",0,2,1,0,0,0,Software y Servicios B2B
4,TripleZip,https://www.crunchbase.com/organization/triplezip,500000.0,0,500000.0,0,2025-01-05,0,0,1,...,NaN,0,"Artificial Intelligence (AI), Data and Analyti...",0,2,1,0,1,16,Bienes Raíces y Construcción


In [26]:
df = df.drop(["categories"], axis=1)
len(df)

2557

In [27]:
# Se trabajará solo con startups activas
df = df[df["operating_status"] != "closed"]
len(df)

2438

In [28]:
#Calcular edad de la compañia

# Asegurar que la columna founded_on esté en formato fecha
df["founded_on"] = pd.to_datetime(df["founded_on"], errors="coerce")

# Fecha de corte para el cálculo
hoy = pd.to_datetime("today")

#fecha ficticia para compañias con estado closed
fecha_cierre_ficticia = pd.to_datetime("2020-12-31")

df["company_age"] = df.apply(
    lambda row: ((fecha_cierre_ficticia if row["operating_status"] == "closed" else hoy) - row["founded_on"]).days // 365
    if pd.notna(row["founded_on"]) else pd.NA,
    axis=1
).astype("Int64")


In [29]:
len(df)

2438

In [30]:
#Se trabajará con startups fundadas entre el 2014 y 2019
df = df[
    (df["founded_on"].dt.year >= 2014) &
    (df["founded_on"].dt.year <= 2019)
]
len(df)

753

In [31]:
# Guardar 
df.to_csv("datasets/startups_etapa_temprana_final.csv", index=False)

In [32]:
# Guardar 

df_procesamiento_manual =  df[["company_name","link_company", "linkedin", "linkedin_employees", "linkedin_followers", "facebook", "facebook_followers"]]

df_procesamiento_manual.to_csv("datasets/startups_etapa_temprana_para_procesamiento_manual.csv", index=False)

In [33]:
len(df)

753